In [58]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from bs4 import BeautifulSoup
import re

driver_path = "driver/geckodriver.exe"
page_url = "https://rmsothebys.com/en/search#/?SortBy=Default&SearchTerm=&Category=All%20Categories&IncludeWithdrawnLots=false&Auction=&OfferStatus=Results&AuctionYear=&Model=Model&Make=Make&FeaturedOnly=false&StillForSaleOnly=false&Collection=All%20Lots&WithoutReserveOnly=false&Day=All%20Days&CategoryTag=All%20Motor%20Vehicles&page=1&pageSize=200&ToYear=NaN&FromYear=NaN"

driver = webdriver.Firefox(executable_path=driver_path)
driver.get(page_url)
WebDriverWait(driver, 10)

<selenium.webdriver.support.wait.WebDriverWait (session="cb99a45a-4d61-4e36-ae5e-a82c3c0198ec")>

In [64]:
html_page = driver.page_source
html_soup = BeautifulSoup(html_page)

car_containers = html_soup.find_all('div', class_="search-result__caption")
search_result = car_containers[0] #0 is the first result of the search, in this case, first car

In [63]:
len(car_containers)

40

In [65]:
search_result

<div class="search-result__caption">
<h5 class="heading-details--bolder ng-binding">
</h5>
<p class="heading-subtitle--bold ng-binding">2006 Ford GT </p>
<p>
<span class="heading-subtitle--bold ng-binding">Sold For $310,500</span><br/>
<span class="heading-subtitle--bold ng-binding ng-hide" ng-show="item.PreSaleEstimate.length &gt; 0"></span><br/>
<span class="heading-details--bolder ng-binding">Offered without reserve</span><br/>
<span class="heading-details--bolder ng-binding ng-hide" ng-show="item.StyleClass == 'RMOnlineAuctions' &amp;&amp; item.CurrentBid"><br/></span>
<span class="heading-details--bolder ng-binding ng-hide" ng-show="item.StyleClass == 'RMOnlineAuctions' &amp;&amp; item.TimeLeft"><br/></span>
</p>
<span class="heading-details--bolder search-result__bottom-left ng-binding">RM | ONLINE ONLY</span>
</div>

## An example of how scraping works

Let's access just the fields we need using find_all method and restricting the result to the order of occurence of the HTML tag within the search result. 

In [71]:
car_info = search_result.find_all('p')[0]
price = search_result.find_all('span')[0] #price can be found in the first span, on the 0th position so to speak
auction_type = search_result.find_all('span')[-1] #auction type can be found in the last span found
auction_location = search_result.find_all('h5')[0]

Now we know how to obtain values that are of interest to us, but we are stuck with HTML code. To get rid of it, we'll use regular expressions. The following expression uses: 
* lookahead to determine characters preceding our pattern,
* a capture group to make accessing our pattern easier,
* lookbehind to determine characters following our pattern.

Because re.search expects it's arguments to be a string, we need to convert our search results to a string.

In [72]:
pattern = r'(?<=">)\s*(.*)\s*(?=</)'

car_re = re.search(pattern, str(car_info), re.IGNORECASE).group(1)
price_re = re.search(pattern, str(price), re.IGNORECASE).group(1)
auct_type_re = re.search(pattern, str(auction_type), re.IGNORECASE).group(1)
auct_loc_re = re.search(pattern, str(auction_location), re.IGNORECASE).group(1)

Let's see the results:

In [68]:
print(car_re, price_re, auct_type_re, auct_loc_re)

2006 Ford GT  Sold For $310,500 RM | ONLINE ONLY 


## Automating the process

Now, there are two ways to go about this:
1. Get the search results, append them to one of the tables, construct a DataFrame object, change the types to String and then apply regular expressions to a DataFrame using vectorised methods or
2. Get the search results, change the types to String, apply regular expressions, then append them to one ofthe tables and construct a DataFrame object.

Common sense suggests, that the first approach would be better, but let's keep in mind, that the raw scrape results are long HTML tags, making everything really hard to read and spotting potential errors even harder. Measuring performance differences between the two approaches is currently not the scope of the project, but can be easily done using e.g. timeit module

In [56]:
def scrape_current_page():
    el_per_page = 
    
def scrape_multiple_pages(number_of_pages, elements_per_page):
    for page in number_of_pages:
        scrape_current_page(elemets_per_page)
        
# applying the regex last, on a finished DataFrame object, using vectorised methods


In [6]:
# when you're finished with scraping, close the webdriver
# wd.quit()